In [ ]:
import pandas as pd
import numpy as np
import collections
import imblearn
import typing

In [ ]:
#Global variables
methods = {'SMOTETomek': imblearn.combine.SMOTETomek, 
           'RandomOver': imblearn.over_sampling.RandomOverSampler,
           'ADASYN': imblearn.over_sampling.ADASYN,
           'RandomUnder': imblearn.under_sampling.RandomUnderSampler,
           'none': None,}

In [ ]:
#Utils
def value_or_zero(key: str, temp_dict: dict[str, int]) -> int:
    '''Return value in a dictionary or zero if key not in dictionary.'''
    return 0 if key not in temp_dict else temp_dict[key]

In [ ]:
#Word representations
def bow(text: str, wordlist: list[str]) -> list[int]:
    '''Represent text with bag of words.'''
    return [int(word in text) for word in wordlist]

def freq(text: str, wordlist: list[str]) -> list[float]:
    '''Represent text with frequencies.''''
    text = text.split()
    counter = collections.Counter(text)
    length = len(text)
    return [value_or_zero(word, counter)/length for word in wordlist]

def tfidf(text: str, word_dict: dict[str, float]) -> list[float]:
    '''Represent text with TF-IDF'''
    text = text.split()
    counter = collections.Counter(text)
    length = len(text)
    frequencies = [value_or_zero(word, counter)/length for word in word_dict]
    return [word_dict[word] * frequencies[i] for (i, word) in enumerate(word_dict)]

In [ ]:
#Primary visualization functions
def visualize_rep_method(inputs: np.ndarray, outputs: np.ndarray) -> None:
    '''Visualize a particular representation and resampling method.'''
    pass

def visualize_method(method: str, method_call: typing.Optional[callable]) -> None:
    '''Visualize each representation using a resampling method.'''
    pass

In [ ]:
def main() -> None:
    for method in methods:
        visualize_method(method, methods[method])

In [ ]:
main()